In [ ]:
from google.cloud import vision
import io
import os
import cv2
import pytesseract
import shutil

# Configuracion de la autenticacion para usar la API de Google
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\TFG\\cosmic-surface-424605-k7-3a347a7f8214.json'

# Configura la ubicacion del ejecutable de tesseract (debe estar instalado en tu sistema)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

muestra_texto = 'C:\\TFG\\muestra_imagenes_texto'
total_imagenes = len(os.listdir(muestra_texto))

carpeta_imagenes = 'C:\\TFG\\imagenes'

def cleanup_text(text):
    return ''.join([c if ord(c) < 128 else '' for c in text]).strip()

Google Cloud Vision API

In [ ]:
imagenes_texto = {}
imagenes_con_texto = 0
client = vision.ImageAnnotatorClient()

for f in os.listdir(muestra_texto):
    image_path = os.path.join(muestra_texto, f)
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
        image = vision.Image(content=content)
        response = client.text_detection(image=image)
        if response.error.message:
            raise Exception(f'{response.error.message}')

    # Mostrar la informacion por pantalla 
    if response.text_annotations:
        porcentaje_texto = response.text_annotations[0].score
        texto = response.text_annotations[0].description
        imagenes_texto[f] = (response.text_annotations[0].score, response.text_annotations[0].description)
        if porcentaje_texto > 0.5: # Si se detecta texto con mas de un 50% de confianza
            imagenes_con_texto += 1

            
# Calcula el porcentaje de imagenes con texto
porcentaje_con_texto = (imagenes_con_texto / total_imagenes) * 100

# Imprime el resultado
print(f"Porcentaje de imagenes con texto: {porcentaje_con_texto:.2f}%")
print(f"Imagenes con texto: {imagenes_con_texto}")
print(f"Imagenes sin texto: {total_imagenes - imagenes_con_texto}")

OPEN CV Y TESSERACT

In [ ]:
# Lista para almacenar las imagenes con texto
im_cont_texto = []

# Itera sobre los archivos en la carpeta
for archivo in os.listdir(carpeta_imagenes):
    ruta_imagen = os.path.join(muestra_texto, archivo)
    # Carga la imagen
    imagen = cv2.imread(ruta_imagen)
    # Convierte la imagen a escala de grises
    i_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    # Aplica OCR a la imagen
    texto = pytesseract.image_to_string(i_gris)
    texto = cleanup_text(texto)
    # Verifica si hay texto en la imagen
    if len(texto) > 0:
        im_cont_texto.append(archivo)
        
        
# Calcula el porcentaje de imagenes con texto
porcentaje_con_texto = (len(im_cont_texto) / total_imagenes) * 100

# Imprime el resultado
print(f"Porcentaje de imagenes con texto: {porcentaje_con_texto:.2f}%")
print(f"Imagenes con texto: {len(im_cont_texto)}")
print(f"Imagenes sin texto: {total_imagenes - len(im_cont_texto)}")

Dividir las imágenes en función de la clasificación con Open CV y Tesseract

In [ ]:
carpeta_con_texto = 'C:\\TFG\\imagenes_con_texto'
carpeta_sin_texto = 'C:\\TFG\\imagenes_sin_texto'
mover_con_texto = set()
mover_sin_texto = set()
copiados_texto = 0
copiados_sin_texto = 0

for imagen in os.listdir(carpeta_imagenes):
    ruta_origen = os.path.join(carpeta_imagenes, imagen)
    if imagen in im_cont_texto:
        mover_con_texto.add(imagen)
        # Copiar el archivo a la carpeta externa
        ruta_destino = os.path.join(carpeta_con_texto, imagen)
        shutil.copy(ruta_origen, ruta_destino)
        copiados_texto += 1
    if imagen not in im_cont_texto:
        mover_sin_texto.add(imagen)
        # Copiar el archivo a la carpeta externa
        ruta_destino = os.path.join(carpeta_sin_texto, imagen)
        shutil.copy(ruta_origen, ruta_destino)
        copiados_sin_texto += 1

print(f"Se han copiado {copiados_texto} archivos a {carpeta_con_texto}.")
print(f"Se han copiado {copiados_sin_texto} archivos a {carpeta_sin_texto}.")